In [40]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [41]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [43]:
def model_input(state_size):
    prev_actions = tf.placeholder(tf.int32, [None], name='prev_actions')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return prev_actions, states, actions, next_states, rewards, dones, rates

In [44]:
def Act(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        actions_logits = tf.layers.dense(inputs=nl2, units=action_size, trainable=False)
        return actions_logits

In [45]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [46]:
def model_loss(state_size, action_size, hidden_size, gamma,
               prev_actions, states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s, a')
    prev_actions_labels = tf.one_hot(indices=prev_actions, depth=action_size, dtype=states.dtype)
    actions_logits = Act(states=states, actions=prev_actions_labels, hidden_size=hidden_size, 
                         action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    e_next_states_logits, eQs_logits = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                           action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs_logits = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                           action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, # GQN
                                                                   labels=next_states_labels))
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs_logits, shape=[-1])
    aQs_logits = tf.reshape(aQs_logits, shape=[-1])
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, actions=actions_labels, hidden_size=hidden_size, 
                              action_size=action_size, reuse=True)
    next_states_logits, aQs2_logits = Env(actions=next_actions_logits, states=next_states, 
                                          hidden_size=hidden_size, action_size=action_size, 
                                          state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2_logits, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a')
    targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
    eloss = tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
    eloss += tf.reduce_mean(aQs_logits) # minimize Q
    aloss2 += -tf.reduce_mean(aQs_logits) # maximize Q
    return actions_logits, aloss, eloss, aloss2

In [47]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [48]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma=gamma):

        # Data of the Model: make the data available inside the framework
        self.prev_actions, self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            prev_actions=self.prev_actions, states=self.states, actions=self.actions, # model input
            next_states=self.next_states, rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [49]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
        return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [50]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [51]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [52]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [53]:
state = env.reset()
total_reward = 0
num_step = 0
prev_action = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([prev_action, state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    prev_action = action
    if done is True:
        #print(done)
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0
        num_step = 0
        prev_action = 0

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list= [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1
        prev_action = 0

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                feed_dict={model.states: state.reshape([1, -1]), model.prev_actions: np.reshape(prev_action, [1])}
                action_logits = sess.run(model.actions_logits, feed_dict)
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([prev_action, state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            prev_action = action
            
            # Training batches
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                prev_actions = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[6] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                prev_actions = prev_actions[rates >= np.max(rates)]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            feed_dict = {model.prev_actions: prev_actions,
                         model.states: states, 
                         model.actions: actions,
                         model.next_states: next_states,
                         model.rewards: rewards,
                         model.dones: dones,
                         model.rates: rates}
            aloss, _, eloss, _ = sess.run([model.a_loss, model.a_opt, model.e_loss, model.e_opt], feed_dict)
            aloss2, _ = sess.run([model.a_loss2, model.a_opt2], feed_dict)
            #print(aloss2, eloss2)
            #print(prev_actions.shape, prev_actions.dtype)
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:34.0000 R:34.0000 rate:0.0680 aloss:0.7313 eloss:0.9880 aloss2:1.5704 exploreP:0.9966
Episode:1 meanR:24.5000 R:15.0000 rate:0.0300 aloss:0.7269 eloss:0.9450 aloss2:1.6159 exploreP:0.9952
Episode:2 meanR:20.6667 R:13.0000 rate:0.0260 aloss:0.7183 eloss:1.0662 aloss2:1.5604 exploreP:0.9939
Episode:3 meanR:20.2500 R:19.0000 rate:0.0380 aloss:0.7263 eloss:0.9303 aloss2:1.5825 exploreP:0.9920
Episode:4 meanR:17.8000 R:8.0000 rate:0.0160 aloss:0.7281 eloss:0.9673 aloss2:1.5910 exploreP:0.9912
Episode:5 meanR:17.6667 R:17.0000 rate:0.0340 aloss:0.7264 eloss:0.8857 aloss2:1.6831 exploreP:0.9896
Episode:6 meanR:18.5714 R:24.0000 rate:0.0480 aloss:0.7199 eloss:0.8584 aloss2:1.6436 exploreP:0.9872
Episode:7 meanR:20.5000 R:34.0000 rate:0.0680 aloss:0.7296 eloss:0.8642 aloss2:1.5882 exploreP:0.9839
Episode:8 meanR:21.1111 R:26.0000 rate:0.0520 aloss:0.7309 eloss:0.7564 aloss2:1.6375 exploreP:0.9814
Episode:9 meanR:20.9000 R:19.0000 rate:0.0380 aloss:0.7290 eloss:0.7114 aloss2:1.68

Episode:80 meanR:21.4938 R:23.0000 rate:0.0460 aloss:0.7286 eloss:0.6202 aloss2:2.1048 exploreP:0.8418
Episode:81 meanR:21.4146 R:15.0000 rate:0.0300 aloss:0.7143 eloss:0.6220 aloss2:2.0642 exploreP:0.8406
Episode:82 meanR:21.3976 R:20.0000 rate:0.0400 aloss:0.7224 eloss:0.6299 aloss2:2.0289 exploreP:0.8389
Episode:83 meanR:21.3095 R:14.0000 rate:0.0280 aloss:0.7237 eloss:0.5745 aloss2:2.2227 exploreP:0.8377
Episode:84 meanR:21.4000 R:29.0000 rate:0.0580 aloss:0.7127 eloss:0.5721 aloss2:2.2353 exploreP:0.8353
Episode:85 meanR:21.6860 R:46.0000 rate:0.0920 aloss:0.7232 eloss:0.5918 aloss2:2.1387 exploreP:0.8316
Episode:86 meanR:21.5862 R:13.0000 rate:0.0260 aloss:0.7205 eloss:0.5699 aloss2:2.2048 exploreP:0.8305
Episode:87 meanR:21.5682 R:20.0000 rate:0.0400 aloss:0.7243 eloss:0.5652 aloss2:2.2082 exploreP:0.8289
Episode:88 meanR:21.5955 R:24.0000 rate:0.0480 aloss:0.7185 eloss:0.5781 aloss2:2.1991 exploreP:0.8269
Episode:89 meanR:21.6333 R:25.0000 rate:0.0500 aloss:0.7154 eloss:0.5882 

Episode:159 meanR:18.2700 R:21.0000 rate:0.0420 aloss:0.7166 eloss:-0.3687 aloss2:3.6742 exploreP:0.7303
Episode:160 meanR:18.2600 R:15.0000 rate:0.0300 aloss:0.7189 eloss:-0.7840 aloss2:4.5282 exploreP:0.7292
Episode:161 meanR:18.1900 R:11.0000 rate:0.0220 aloss:0.7085 eloss:-0.5911 aloss2:4.0413 exploreP:0.7284
Episode:162 meanR:18.3300 R:23.0000 rate:0.0460 aloss:0.7107 eloss:-0.5389 aloss2:3.9272 exploreP:0.7267
Episode:163 meanR:18.3000 R:23.0000 rate:0.0460 aloss:0.7087 eloss:-0.5345 aloss2:3.9182 exploreP:0.7251
Episode:164 meanR:18.4300 R:24.0000 rate:0.0480 aloss:0.7129 eloss:-0.7729 aloss2:4.4771 exploreP:0.7234
Episode:165 meanR:18.2800 R:12.0000 rate:0.0240 aloss:0.7178 eloss:-0.6023 aloss2:3.9672 exploreP:0.7225
Episode:166 meanR:18.3700 R:22.0000 rate:0.0440 aloss:0.7094 eloss:-0.6526 aloss2:4.0488 exploreP:0.7210
Episode:167 meanR:18.2700 R:17.0000 rate:0.0340 aloss:0.7158 eloss:-0.7227 aloss2:4.2509 exploreP:0.7197
Episode:168 meanR:18.2800 R:12.0000 rate:0.0240 aloss:0

Episode:237 meanR:42.3100 R:94.0000 rate:0.1880 aloss:0.6992 eloss:-5.6020 aloss2:17.3228 exploreP:0.4999
Episode:238 meanR:42.5000 R:41.0000 rate:0.0820 aloss:0.6944 eloss:-4.6760 aloss2:17.6475 exploreP:0.4978
Episode:239 meanR:42.6700 R:40.0000 rate:0.0800 aloss:0.7089 eloss:-5.5942 aloss2:17.3781 exploreP:0.4959
Episode:240 meanR:43.7100 R:116.0000 rate:0.2320 aloss:0.7064 eloss:-5.4569 aloss2:17.9548 exploreP:0.4903
Episode:241 meanR:44.8600 R:128.0000 rate:0.2560 aloss:0.7141 eloss:-5.4943 aloss2:18.9607 exploreP:0.4842
Episode:242 meanR:45.8100 R:110.0000 rate:0.2200 aloss:0.7238 eloss:-5.8545 aloss2:18.7016 exploreP:0.4790
Episode:243 meanR:46.6400 R:103.0000 rate:0.2060 aloss:0.7292 eloss:-6.0280 aloss2:20.0692 exploreP:0.4742
Episode:244 meanR:47.0700 R:59.0000 rate:0.1180 aloss:0.7344 eloss:-6.1872 aloss2:20.1688 exploreP:0.4715
Episode:245 meanR:47.9600 R:107.0000 rate:0.2140 aloss:0.7121 eloss:-6.2399 aloss2:20.6078 exploreP:0.4666
Episode:246 meanR:48.7400 R:91.0000 rate:

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
